### 데이터 불균형 : 클래스가 어느 한 쪽으로만 일방적으로 존재

* 해결 방법
1) 오버 샘플링 : 
>클래스가 적은 쪽의 데이터를 랜덤샘플링하여 복사 붙여넣기를 반복하여 
                    두 클래스의 비율을 비슷하게 함.
                          
2) 언더 샘플링 : 
>클래스가 많은 쪽의 데이터를 랜덤 샘플링하여 삭제하기를 반복하여 
                    두 클래스의 비율을 비슷하게 함.

3) 오버 & 언더 샘플링
4) SMOTE 알고리즘
 >기존 데이터를 적절하게 혼합하여 새로운 데이터를 생성하는 방법

> ex) Y : 1000건 vs N : 10건 -> 1010 / 2 = 505, 
>> Y는 505건이 될때까지 언더 샘플링
>> N은 505건이 될때까지 오버 샘플링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
card_df = pd.read_csv("creditcard.csv")
card_df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


* train 데이터와 test 데이터 나누기

In [5]:
from sklearn.model_selection import train_test_split

In [7]:
x_features = card_df.iloc[:,:-1]  # x
y_target = card_df.iloc[:,-1]   # y 구하고자 하는 값 "class"

In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(x_features,y_target,
                                                test_size = 0.3, random_state=20231023,
                                               stratify=y_target # 층화추출 : 비율 맞춰 추출
                                               )

In [16]:
ytrain.value_counts() # 비정상 거래 336건

0    199020
1       344
Name: Class, dtype: int64

In [17]:
ytest.value_counts()

0    85295
1      148
Name: Class, dtype: int64

In [20]:
def get_preprocessed_df(df = None):
    df_copy = df.copy()
    df_copy.drop("Time",axis = 1, inplace = True)
    return df_copy

In [21]:
df_copy = get_preprocessed_df(card_df)

In [23]:
df_copy

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [24]:
def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

In [25]:
print(y_train.value_counts()/y_train.shape[0]*100)

0    99.827451
1     0.172549
Name: Class, dtype: float64


In [26]:
print(y_test.value_counts()/y_test.shape[0]*100)

0    99.826785
1     0.173215
Name: Class, dtype: float64


In [32]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

In [34]:
import warnings
warnings.filterwarnings("ignore")
lr_clf.fit(X_train,y_train)

LogisticRegression()

In [37]:
lr_clf.predict(X_test)
pd.Series(lr_clf.predict(X_test)).value_counts()

0    85341
1      102
dtype: int64

In [39]:
lr_clf.predict_proba(X_test)
lr_clf.predict_proba(X_test)[:,1]

array([0.00134831, 0.00012345, 0.00019178, ..., 0.00024669, 0.00074666,
       0.00013475])